# テキストデータ解析スクリプト

このスクリプトは、指定したディレクトリ内のテキストファイルを読み込み、spaCyライブラリを使用して言語解析を行う。解析結果はDocオブジェクトとして保存され、他のスクリプトで再利用できる。

## 使用方法

1. 解析対象のテキストファイルを格納したディレクトリ（フォルダ）を用意する。
2. このスクリプトを開き、カーネルに仮想環境を設定する。
3. spaCyライブラリと解析する言語のモデルを、必要に応じて事前にインストールする。
 - 例: `!pip install spacy` , `!python -m spacy download en_core_web_sm`
4. スクリプト内の「ユーザーが編集する設定」セクションで以下の変数を設定する。
 - `input_directory`: 解析対象のテキストファイルが格納されたディレクトリ名
 - `nlp_model`: 解析に使用するモデルの名前（例: "en_core_web_sm"）
 - `output_subdirectory`: 今回の解析結果を保存する子ディレクトリ名
 - `output_main_directory`: 子ディレクトリが格納される親ディレクトリ名
 - `encoding`: テキストファイルのエンコーディング（デフォルトは "utf-8"）
5. スクリプトを実行する。

## 出力結果

設定した出力ディレクトリ(output_subdirectory)に以下の2つのファイルが生成される。

- `processed_YYYYMMDD_HHMMSS.spacy`: 解析結果のDocオブジェクトが格納されたバイナリファイル
- `processed_YYYYMMDD_HHMMSS.txt`: 処理したファイル名のリストが格納されたテキストファイル

生成されたファイル名には、処理が実行された日時が含まれる。

## 追加機能

- チャンク単位の解析: 大きなテキストファイルを小さなチャンク（行）に分割して処理できる。
- spacy-stanzaを用いた解析: spacy-stanzaパッケージを使用して、Stanzaのモデルで解析を行える。

# **1. テキスト解析＆保存**  
## ユーザーが設定する項目

In [4]:
input_directory = "finance"  # 解析対象のテキストファイルが保存されたディレクトリへのパス
nlp_model = "en_core_web_sm"  # 解析に使用するモデルの名前
output_subdirectory = "user_defined_subdir"  # 今回の解析結果を保存する子ディレクトリ名
output_main_directory = "processed_data"  # 親ディレクトリ名
encoding = "utf-8"  # テキストファイルのエンコーディング

## テキスト解析の実行

In [ ]:
import os
import spacy
from spacy.tokens import DocBin
from datetime import datetime

def load_text(file_path, encoding='utf-8'):
    """
    テキストファイルを読み込む関数

    Args:
        file_path (str): 読み込むテキストファイルのパス
        encoding (str, optional): テキストファイルのエンコーディング. デフォルトは 'utf-8'.

    Returns:
        str: 読み込んだテキストファイルの内容（改行をスペースに置換済み）
    """
    with open(file_path, 'r', encoding=encoding) as f:
        return f.read().replace('\n', ' ')  # 改行をスペースに置換して返す(注意)

nlp = spacy.load(nlp_model)  # spaCyの言語モデルをロード
output_directory = os.path.join(output_main_directory, output_subdirectory)  # メインディレクトリとサブディレクトリのパスを作成
os.makedirs(output_directory, exist_ok=True)  # メインディレクトリとサブディレクトリを作成（存在しない場合）

doc_bin = DocBin()  # DocBinクラスのインスタンスを作成
file_names = []  # ファイル名を保存するための空のリストを作成
now_str = datetime.now().strftime("%Y%m%d_%H%M%S")  # 現在の日時を取得してファイル名に組み込むための文字列を生成

# テキストファイルを読み込んでDocオブジェクトに変換し、DocBinに追加
for filename in sorted(os.listdir(input_directory)):  # ファイル名でソートしてループ
    if filename.endswith(".txt"):
        file_path = os.path.join(input_directory, filename)
        text = load_text(file_path, encoding)  # テキストファイルを読み込む
        doc = nlp(text)  
        
        doc_bin.add(doc)
        file_names.append(filename)
        
        print(f"Processed: {filename}")

# DocBinをサブディレクトリに保存
doc_bin_file_path = os.path.join(output_directory, f"processed_{now_str}.spacy")
doc_bin.to_disk(doc_bin_file_path)

# ファイル名をサブディレクトリに保存
file_names_path = os.path.join(output_directory, f"processed_{now_str}.txt")
with open(file_names_path, 'w') as f:
    for name in file_names:
        f.write(name + '\n')

print(f"全てのDocオブジェクトが '{doc_bin_file_path}' に保存されました。")
print(f"ファイル名が '{file_names_path}' に保存されました。")

# **2. チャンク単位の解析**
## ユーザーが設定する項目

In [1]:
input_directory = "jp_dowa"  # 解析対象のテキストファイルが保存されたディレクトリ名
nlp_model = "ja_ginza"  # 解析に使用するモデルの名前
chunk_size = 100  # テキストファイルを分割する単位（行数）

output_subdirectory = "user_defined_subdir"  # 今回の解析結果を保存する子ディレクトリ名
output_main_directory = "processed_data"  # 親ディレクトリ名
encoding = "utf-8"  # テキストファイルのエンコーディング

## テキスト解析の実行

In [ ]:
import os
import spacy
from spacy.tokens import DocBin
from datetime import datetime

def load_text(file_path, encoding='utf-8'):
    """
    テキストファイルを読み込む関数

    Args:
        file_path (str): 読み込むテキストファイルのパス
        encoding (str, optional): テキストファイルのエンコーディング. デフォルトは 'utf-8'.

    Returns:
        list: 読み込んだテキストファイルの内容（改行で分割済み）
    """
    with open(file_path, 'r', encoding=encoding) as f:
        return f.readlines()  # 改行ごとにリストに格納して返す

nlp = spacy.load(nlp_model)  # spaCyの言語モデルをロード
output_directory = os.path.join(output_main_directory, output_subdirectory)  # メインディレクトリとサブディレクトリのパスを作成
os.makedirs(output_directory, exist_ok=True)  # メインディレクトリとサブディレクトリを作成（存在しない場合）

doc_bin = DocBin()  # DocBinクラスのインスタンスを作成
file_names = []  # ファイル名を保存するための空のリストを作成
now_str = datetime.now().strftime("%Y%m%d_%H%M%S")  # 現在の日時を取得してファイル名に組み込むための文字列を生成

for filename in sorted(os.listdir(input_directory)):  
    if filename.endswith(".txt"):
        file_path = os.path.join(input_directory, filename)
        lines = load_text(file_path, encoding)  # テキストファイルを読み込む(改行ごとにリストに格納)
        
        for i in range(0, len(lines), chunk_size):  # chunk_sizeの値だけループ変数 i が増加
            chunk_text = ''.join(lines[i:i+chunk_size])  # chunk_sizeごとにテキストを結合
            chunk_text = chunk_text.replace('\n', ' ')  # 改行をスペースに置換(注意)
            doc = nlp(chunk_text)
            
            doc_bin.add(doc)
            file_names.append(filename + f"_chunk_{i//chunk_size + 1}") # ファイル名にチャンク番号を追加
            
            print(f"Processed: {filename}, chunk {i//chunk_size + 1}")

# DocBinをサブディレクトリに保存（日時をファイル名に組み込む）
doc_bin_file_path = os.path.join(output_directory, f"processed_{now_str}.spacy")
doc_bin.to_disk(doc_bin_file_path)

# ファイル名をサブディレクトリに保存（日時をファイル名に組み込む）
file_names_path = os.path.join(output_directory, f"processed_{now_str}.txt")
with open(file_names_path, 'w') as f:
    for name in file_names:
        f.write(name + '\n')

print(f"全てのDocオブジェクトが '{doc_bin_file_path}' に保存されました。")
print(f"ファイル名が '{file_names_path}' に保存されました。")

# **3. spacy-stanzaを用いた解析**
## ユーザーが設定する項目

In [1]:
input_directory = "finance"  # 解析対象のテキストファイルが保存されたディレクトリ名
nlp_model = "en"  # 解析に使用するモデルの名前(stanzaのモデル)
output_subdirectory = "user_defined_subdir"  # 今回の解析結果を保存する子ディレクトリ名
output_main_directory = "processed_data"  # 親ディレクトリ名
encoding = "utf-8"  # テキストファイルのエンコーディング

## テキスト解析の実行

In [ ]:
import os
import spacy_stanza
from spacy.tokens import DocBin
from datetime import datetime

def load_text(file_path, encoding='utf-8'):
    """
    テキストファイルを読み込む関数

    Args:
        file_path (str): 読み込むテキストファイルのパス
        encoding (str, optional): テキストファイルのエンコーディング. デフォルトは 'utf-8'.

    Returns:
        str: 読み込んだテキストファイルの内容（改行をスペースに置換済み）
    """
    with open(file_path, 'r', encoding=encoding) as f:
        return f.read().replace('\n', ' ')  # 改行をスペースに置換して返す(注意)

nlp = spacy_stanza.load_pipeline(nlp_model)  # stanzaの言語モデルをロード（必要に応じてダウンロードが実行される）
output_directory = os.path.join(output_main_directory, output_subdirectory)  # メインディレクトリとサブディレクトリのパスを作成
os.makedirs(output_directory, exist_ok=True)  # メインディレクトリとサブディレクトリを作成（存在しない場合）

doc_bin = DocBin()  # DocBinクラスのインスタンスを作成
file_names = []  # ファイル名を保存するための空のリストを作成
now_str = datetime.now().strftime("%Y%m%d_%H%M%S")  # 現在の日時を取得してファイル名に組み込むための文字列を生成

# テキストファイルを読み込んでDocオブジェクトに変換し、DocBinに追加
for filename in sorted(os.listdir(input_directory)):
    if filename.endswith(".txt"):
        file_path = os.path.join(input_directory, filename)
        text = load_text(file_path, encoding)
        doc = nlp(text)
        
        doc_bin.add(doc)
        file_names.append(filename)
        
        print(f"Processed: {filename}")

# DocBinをサブディレクトリに保存（日時をファイル名に組み込む）
doc_bin_file_path = os.path.join(output_directory, f"processed_{now_str}.spacy")
doc_bin.to_disk(doc_bin_file_path)

# ファイル名をサブディレクトリに保存（日時をファイル名に組み込む）
file_names_path = os.path.join(output_directory, f"processed_{now_str}.txt")
with open(file_names_path, 'w') as f:
    for name in file_names:
        f.write(name + '\n')

print(f"全てのDocオブジェクトが '{doc_bin_file_path}' に保存されました。")
print(f"ファイル名が '{file_names_path}' に保存されました。")